In [1]:
# %load_ext autotime
# !pip install autotime
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import warnings
import pandas as pd
import numpy as np
import time
import esm
import biotite.structure.io as bsio
from proteinttt.models.esmfold import ESMFoldTTT, DEFAULT_ESMFOLD_TTT_CFG
from proteinttt.utils.structure import calculate_tm_score, lddt_score
import torch
import argparse
import os


pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 1000)

/scratch/project/open-35-8/pimenol1/miniconda3/envs/proteinttt/lib/python3.10/site-packages/lightning_lite/__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)


In [2]:
base_path = Path("/scratch/project/open-35-8/pimenol1/ProteinTTT/ProteinTTT/data/bfvd/")
SUMMARY_PATH = base_path / 'proteinttt_results.tsv'
MSA_PATH = Path("/scratch/project/open-35-8/antonb/bfvd/bfvd_msa")
SUBSET_PATH = base_path / 'proteinttt_msa_testset.tsv'

# Choosing data

In [14]:
df = pd.read_csv(SUMMARY_PATH, sep="\t")
df.head(2)

,id,nmsa,pLDDT_AlphaFold,version,sequence,length,pLDDT_before,pLDDT_after,tm_score_before,lddt_before,tm_score_after,lddt_after,time,difference_plddt
0,A7IUX3,3,40.184184,BASE,MYKMSLLVLTQEFQEEDMPWFGTAFLHPHVPNRGPCPSFHLDNPKNLLWVCEWYTCIYLAWDQDNQYTLGMMHTEIRRIHLPSHDRCLGFPGDTSMCCQLGLS,103,28.709551,36.696726,0.28594,0.331706,0.33734,0.350680,36.788191,7.987175
1,A0A6M9TZY3,7,40.185052,BASE+LOGAN+12CY,MLLDKPDQHVITEFDAFMENDKIILNGIFNDTAKKALNLSLPLGFFSISNYIYFNLPLLEFPSFVNNYTFKDPLHKERIHYSFELSNVYNNLQTDYFNNVIRHPLPKTKPVITGIKVLPKITRSIEGPSSRINNSDKDFNHREVHLERKKPYDKKPNMRETDQQNMPPPPLPKMKKIKTTSAESAAAPSSSQPANINDIDAHLQKYHQLFTKLIQEKEFKKYQKFVDDFKVFVTPIYHVIQANTLKYKPLIETYNRYVLECFVRHWKIKTTDNLKSSLHAISNTEYDNFIASYHNFNEMYITLNDMSNIKEDPDYTIIKEFVHEIVRLIGINNNI,335,42.312089,58.246706,0.43240,0.533388,0.39168,0.544333,185.468657,15.934617


In [28]:
df1 = df.query('pLDDT_AlphaFold > 90').sample(250)

In [30]:
df2 = df1.query('pLDDT_before <50')
df3 = df1.query('pLDDT_before >50')

df_r = pd.concat([df2.sample(40), df3.sample(60)])

In [37]:
df_r.to_csv(SUBSET_PATH, sep="\t", index=False)

In [7]:
df['pLDDT_0.0004_4'].describe()

count    97.000000
mean     74.738417
std      14.479082
min      32.623285
25%      68.554261
50%      77.803222
75%      85.509580
max      93.753057
Name: pLDDT_0.0004_4, dtype: float64

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob

def plot_mean_plddt_vs_step(logs_dir, hyperparams=None, output_path=None):
    """
    Create plots showing mean plddt across all proteins at each step, 
    organized by hyperparameter combinations.
    
    Args:
        logs_dir: Path to directory containing log files (e.g., 'logs_msa_0.0004_4')
        hyperparams: Optional dict with parameter mappings if you want to organize by params
                    e.g., {'lr': [4e-5, 4e-4, 4e-3], 'ags': [4, 8, 16, 32]}
        output_path: Optional path to save the figure
    """
    
    # Read all log files
    log_files = glob.glob(str(Path(logs_dir) / '*_log.tsv'))
    
    if not log_files:
        print(f"No log files found in {logs_dir}")
        return
    
    # Collect data from all log files
    all_data = []
    for log_file in log_files:
        try:
            # Read with quoting to handle multiline pdb column
            df = pd.read_csv(log_file, sep='\t', quoting=1)  # QUOTE_MINIMAL
            all_data.append(df)
        except Exception as e:
            print(f"Error reading {log_file}: {e}")
            continue
    
    if not all_data:
        print("No valid log files could be read")
        return
    
    # Combine all dataframes
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Calculate mean plddt for each step across all proteins
    mean_plddt = combined_df.groupby('step')['plddt'].mean()
    
    # If hyperparams are provided, create subplots for each combination
    if hyperparams:
        # Extract parameter values from hyperparams dict
        param1_name = list(hyperparams.keys())[0]
        param1_values = hyperparams[param1_name]
        param2_name = list(hyperparams.keys())[1]
        param2_values = hyperparams[param2_name]
        
        nrows, ncols = len(param2_values), len(param1_values)
        fig, axes = plt.subplots(nrows, ncols, figsize=(15, 12))
        
        # Flatten axes for easier iteration
        axes = axes.flatten() if nrows > 1 or ncols > 1 else [axes]
        
        for i, (lr, ags) in enumerate([(lr, ags) for ags in param2_values for lr in param1_values]):
            # Filter data for this hyperparameter combination
            subset_df = combined_df[combined_df['step'].notna()]  # Placeholder - you'd need to filter by actual hyperparams
            mean_plddt_subset = subset_df.groupby('step')['plddt'].mean()
            
            ax = axes[i]
            ax.plot(mean_plddt_subset.index, mean_plddt_subset.values, 
                   'o-', color='green', linewidth=2, markersize=5, label='Mean pLDDT')
            ax.set_xlabel('Step', fontsize=10)
            ax.set_ylabel('Mean pLDDT', fontsize=10, color='green')
            ax.tick_params(axis='y', labelcolor='green')
            ax.set_title(f'{param1_name}: {lr}, {param2_name}: {ags}', fontsize=9)
            ax.grid(True, alpha=0.3)
            ax.legend(loc='upper left')
        
        plt.tight_layout()
    else:
        # Simple single plot
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(mean_plddt.index, mean_plddt.values, 
               'o-', color='green', linewidth=2, markersize=5, label='Mean pLDDT')
        ax.set_xlabel('Step', fontsize=12)
        ax.set_ylabel('Mean pLDDT', fontsize=12, color='green')
        ax.tick_params(axis='y', labelcolor='green')
        ax.set_title('Mean pLDDT vs Step (All Proteins)', fontsize=14)
        ax.grid(True, alpha=0.3)
        ax.legend(loc='best')
        plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
    return mean_plddt


# Example usage:
if __name__ == "__main__":
    from pathlib import Path
    
    base_path = Path("/scratch/project/open-35-8/pimenol1/ProteinTTT/ProteinTTT/data/bfvd/")
    
    # Simple plot for a specific log directory
    plot_mean_plddt_vs_step(base_path / 'logs_msa_0.0004_4')
    
    # Or with hyperparameter organization
    hyperparams = {
        'lr': [4e-5, 4e-4, 4e-3],
        'ags': [4, 8, 16, 32]
    }
    # plot_mean_plddt_vs_step(base_path, hyperparams=hyperparams)

,step,accumulated_step,loss,score_seq_time,eval_step_time,plddt,pdb
0,0,0,NaN,0.0,0.896465,56.824909,PARENT N/A\nATOM 1 N MET A 1 6.278 11.512 -0.976 1.00 48.36 N \nATOM 2 CA MET A 1 7.444 10.711 -1.337 1.00 48.81 C \nATOM 3 C MET A 1 7.021 9.354 -1.891 1.00 49.58 C \nATOM 4 CB MET A 1 8.361 10.519 -0.128 1.00 44.87 C \nATOM 5 O MET A 1 6.069 8.746 -1.396 1.00 48.07 O \nATOM 6 CG MET A 1 9.826 10.341 -0.492 1.00 43.01 C \nATOM 7 SD MET A 1 10.908 10.221 0.985 1.00 50.26 S \nATOM 8 CE MET A 1 11.333 8.458 0.928 1.00 40.17 C \nATOM 9 N SER A 2 7.295 9.083 -3.179 1.00 52.88 N \nATOM 10 CA SER A 2 6.908 7.842 -3.843 1.00 52.61 C \nATOM 11 C SER A 2 8.014 6.797 -3.748 1.00 53.69 C \nATOM 12 CB SER A 2 6.566 8.103 -5.310 1.00 50.52 C \n...
1,1,2,2.378906,0.0,0.901289,88.542816,PARENT N/A\nATOM 1 N MET A 1 13.045 -4.260 3.781 1.00 84.54 N \nATOM 2 CA MET A 1 12.433 -5.395 4.466 1.00 86.60 C \nATOM 3 C MET A 1 11.079 -5.734 3.854 1.00 89.92 C \nATOM 4 CB MET A 1 12.273 -5.100 5.959 1.00 82.19 C \nATOM 5 O MET A 1 10.311 -4.838 3.498 1.00 89.25 O \nATOM 6 CG MET A 1 11.711 -6.265 6.757 1.00 71.12 C \nATOM 7 SD MET A 1 11.628 -5.910 8.555 1.00 66.55 S \nATOM 8 CE MET A 1 10.520 -4.473 8.547 1.00 61.84 C \nATOM 9 N SER A 2 10.803 -7.009 3.620 1.00 90.84 N \nATOM 10 CA SER A 2 9.534 -7.495 3.086 1.00 92.15 C \nATOM 11 C SER A 2 8.510 -7.701 4.197 1.00 91.80 C \nATOM 12 CB SER A 2 9.740 -8.804 2.322 1.00 88.99 C \n...
2,2,4,2.716797,0.0,0.897574,57.763851,PARENT N/A\nATOM 1 N MET A 1 3.945 12.541 -0.458 1.00 51.57 N \nATOM 2 CA MET A 1 5.191 12.006 -1.000 1.00 52.10 C \nATOM 3 C MET A 1 4.984 10.602 -1.558 1.00 52.77 C \nATOM 4 CB MET A 1 6.281 11.987 0.074 1.00 48.03 C \nATOM 5 O MET A 1 4.199 9.823 -1.013 1.00 50.98 O \nATOM 6 CG MET A 1 7.693 11.964 -0.487 1.00 46.20 C \nATOM 7 SD MET A 1 8.972 12.103 0.822 1.00 52.05 S \nATOM 8 CE MET A 1 10.079 10.752 0.333 1.00 43.44 C \nATOM 9 N SER A 2 5.350 10.361 -2.850 1.00 55.59 N \nATOM 10 CA SER A 2 5.116 9.089 -3.527 1.00 55.23 C \nATOM 11 C SER A 2 6.397 8.267 -3.616 1.00 56.43 C \nATOM 12 CB SER A 2 4.552 9.323 -4.929 1.00 53.14 C \n...
3,3,6,2.359375,0.0,0.897136,66.626373,PARENT N/A\nATOM 1 N MET A 1 7.899 12.151 0.100 1.00 65.74 N \nATOM 2 CA MET A 1 9.035 11.295 -0.231 1.00 67.81 C \nATOM 3 C MET A 1 8.651 10.267 -1.289 1.00 68.67 C \nATOM 4 CB MET A 1 9.557 10.587 1.021 1.00 60.37 C \nATOM 5 O MET A 1 7.528 9.758 -1.289 1.00 64.69 O \nATOM 6 CG MET A 1 10.830 9.789 0.787 1.00 54.82 C \nATOM 7 SD MET A 1 11.504 9.072 2.336 1.00 56.52 S \nATOM 8 CE MET A 1 10.509 7.558 2.447 1.00 46.90 C \nATOM 9 N SER A 2 9.419 10.094 -2.315 1.00 70.62 N \nATOM 10 CA SER A 2 9.133 9.169 -3.407 1.00 71.66 C \nATOM 11 C SER A 2 9.982 7.907 -3.298 1.00 71.98 C \nATOM 12 CB SER A 2 9.377 9.842 -4.758 1.00 67.29 C \n...
4,4,8,2.732422,0.0,0.893203,69.801064,PARENT N/A\nATOM 1 N MET A 1 9.687 10.562 -0.684 1.00 68.60 N \nATOM 2 CA MET A 1 10.689 9.546 -0.994 1.00 71.40 C \nATOM 3 C MET A 1 10.148 8.537 -2.000 1.00 72.28 C \nATOM 4 CB MET A 1 11.137 8.827 0.280 1.00 63.38 C \nATOM 5 O MET A 1 8.962 8.204 -1.975 1.00 68.14 O \nATOM 6 CG MET A 1 12.262 7.829 0.057 1.00 56.89 C \nATOM 7 SD MET A 1 12.842 7.062 1.620 1.00 59.00 S \nATOM 8 CE MET A 1 11.463 5.930 1.952 1.00 48.67 C \nATOM 9 N SER A 2 10.897 8.175 -3.002 1.00 73.83 N \nATOM 10 CA SER A 2 10.470 7.236 -4.035 1.00 75.76 C \nATOM 11 C SER A 2 11.140 5.878 -3.858 1.00 75.52 C \nATOM 12 CB SER A 2 10.781 7.790 -5.426 1.00 70.45 C \n...
5,5,10,2.464844,0.0,0.892166,69.855995,PARENT N/A\nATOM 1 N MET A 1 8.444 11.145 0.630 1.00 68.34 N \nATOM 2 CA MET A 1 9.570 10.266 0.329 1.00 71.15 C \nATOM 3 C MET A 1 9.214 9.288 -0.786 1.00 72.15 C \nATOM 4 CB MET A 1 10.004 9.499 1.579 1.00 63.03 C \nATOM 5 O MET A 1 8.073 8.830 -0.875 1.00 68.04 O \nATOM 6 CG MET A 1 11.244 8.644 1.373 1.00 56.57 C \nATOM 7 SD MET A 1 11.797 7.824 2.919 1.00 59.05 S \nATOM 8 CE MET A 1 10.544 6.517 3.052 1.00 48.43 C \nATOM 9 N SER A 2 10.062 9.098 -1.750 1.00 73.78 N \nATOM 10 CA 

In [6]:
log = pd.read_csv(base_path / 'logs_df' / 'A0A1S5R1Q9_log.tsv', sep="\t")
log

,step,accumulated_step,loss,score_seq_time,eval_step_time,plddt,pdb
0,0,0,NaN,0.0,0.831112,58.079447,PARENT N/A\nATOM 1 N MET A 1 4.124 11.013 0.023 1.00 51.28 N \nATOM 2 CA MET A 1 5.332 10.461 -0.583 1.00 52.09 C \nATOM 3 C MET A 1 5.087 9.043 -1.089 1.00 52.83 C \nATOM 4 CB MET A 1 6.487 10.465 0.420 1.00 47.32 C \nATOM 5 O MET A 1 4.320 8.289 -0.489 1.00 50.89 O \nATOM 6 CG MET A 1 7.861 10.524 -0.228 1.00 45.04 C \nATOM 7 SD MET A 1 9.214 10.664 1.004 1.00 51.77 S \nATOM 8 CE MET A 1 10.158 9.164 0.614 1.00 41.51 C \nATOM 9 N SER A 2 5.302 8.769 -2.364 1.00 55.37 N \nATOM 10 CA SER A 2 5.020 7.483 -2.994 1.00 55.17 C \nATOM 11 C SER A 2 6.301 6.687 -3.223 1.00 56.51 C \nATOM 12 CB SER A 2 4.291 7.684 -4.323 1.00 52.95 C \n...
1,1,4,2.412109,0.0,0.830936,77.051393,PARENT N/A\nATOM 1 N MET A 1 8.152 -3.251 -3.175 1.00 72.41 N \nATOM 2 CA MET A 1 7.914 -4.572 -3.751 1.00 75.78 C \nATOM 3 C MET A 1 6.649 -4.574 -4.603 1.00 77.62 C \nATOM 4 CB MET A 1 7.806 -5.628 -2.650 1.00 68.62 C \nATOM 5 O MET A 1 5.689 -3.863 -4.298 1.00 73.95 O \nATOM 6 CG MET A 1 7.660 -7.047 -3.175 1.00 61.75 C \nATOM 7 SD MET A 1 7.663 -8.297 -1.831 1.00 63.89 S \nATOM 8 CE MET A 1 6.149 -7.824 -0.950 1.00 52.91 C \nATOM 9 N SER A 2 6.651 -5.109 -5.715 1.00 76.70 N \nATOM 10 CA SER A 2 5.529 -5.147 -6.647 1.00 79.32 C \nATOM 11 C SER A 2 4.917 -6.541 -6.719 1.00 79.44 C \nATOM 12 CB SER A 2 5.976 -4.705 -8.042 1.00 73.66 C \n...
2,2,8,2.025391,0.0,0.823701,72.327595,PARENT N/A\nATOM 1 N MET A 1 10.159 -7.829 2.264 1.00 70.77 N \nATOM 2 CA MET A 1 9.372 -9.059 2.280 1.00 73.58 C \nATOM 3 C MET A 1 8.330 -9.053 1.167 1.00 74.82 C \nATOM 4 CB MET A 1 8.688 -9.244 3.636 1.00 65.89 C \nATOM 5 O MET A 1 7.770 -8.004 0.841 1.00 70.65 O \nATOM 6 CG MET A 1 7.920 -10.549 3.763 1.00 58.85 C \nATOM 7 SD MET A 1 7.179 -10.770 5.426 1.00 60.50 S \nATOM 8 CE MET A 1 5.611 -9.885 5.198 1.00 49.74 C \nATOM 9 N SER A 2 8.182 -10.116 0.470 1.00 76.31 N \nATOM 10 CA SER A 2 7.220 -10.232 -0.620 1.00 78.49 C \nATOM 11 C SER A 2 6.012 -11.066 -0.205 1.00 78.30 C \nATOM 12 CB SER A 2 7.879 -10.853 -1.853 1.00 73.14 C \n...
3,3,12,1.980469,0.0,0.825270,78.295115,PARENT N/A\nATOM 1 N MET A 1 4.305 3.155 -6.099 1.00 68.76 N \nATOM 2 CA MET A 1 4.448 2.272 -7.253 1.00 71.29 C \nATOM 3 C MET A 1 3.088 1.764 -7.721 1.00 72.79 C \nATOM 4 CB MET A 1 5.360 1.091 -6.917 1.00 63.95 C \nATOM 5 O MET A 1 2.208 1.491 -6.904 1.00 68.18 O \nATOM 6 CG MET A 1 5.785 0.280 -8.131 1.00 58.10 C \nATOM 7 SD MET A 1 7.009 -1.021 -7.713 1.00 60.04 S \nATOM 8 CE MET A 1 8.540 -0.201 -8.240 1.00 51.50 C \nATOM 9 N SER A 2 2.733 1.994 -8.826 1.00 73.56 N \nATOM 10 CA SER A 2 1.471 1.555 -9.413 1.00 74.87 C \nATOM 11 C SER A 2 1.638 0.237 -10.161 1.00 75.30 C \nATOM 12 CB SER A 2 0.920 2.622 -10.360 1.00 70.11 C \n...
4,4,16,2.054688,0.0,0.826880,65.117290,PARENT N/A\nATOM 1 N MET A 1 8.064 5.712 -0.384 1.00 55.76 N \nATOM 2 CA MET A 1 8.532 4.360 -0.677 1.00 55.84 C \nATOM 3 C MET A 1 7.400 3.349 -0.527 1.00 58.06 C \nATOM 4 CB MET A 1 9.695 3.982 0.241 1.00 50.58 C \nATOM 5 O MET A 1 6.619 3.421 0.423 1.00 54.93 O \nATOM 6 CG MET A 1 10.618 2.924 -0.342 1.00 47.53 C \nATOM 7 SD MET A 1 12.077 2.600 0.722 1.00 52.07 S \nATOM 8 CE MET A 1 12.864 1.259 -0.214 1.00 44.81 C \nATOM 9 N SER A 2 6.980 2.658 -1.580 1.00 62.11 N \nATOM 10 CA SER A 2 5.863 1.721 -1.637 1.00 62.09 C \nATOM 11 C SER A 2 6.353 0.286 -1.803 1.00 63.38 C \nATOM 12 CB SER A 2 4.917 2.081 -2.783 1.00 59.38 C \n...
5,5,20,1.514648,0.0,0.831136,70.934065,PARENT N/A\nATOM 1 N MET A 1 6.648 -5.700 -8.319 1.00 59.48 N \nATOM 2 CA MET A 1 5.660 -6.677 -8.766 1.00 60.12 C \nATOM 3 C MET A 1 4.368 -5.988 -9.192 1.00 60.87 C \nATOM 4 CB MET A 1 5.370 -7.694 -7.661 1.00 54.88 C \nATOM 5 O MET A 1 3.934 -5.025 -8.558 1.00 57.44 O \nATOM 6 CG MET A 1 4.998 -9.073 -8.180 1.00 52.01 C \nATOM 7 SD MET A 1 4.780 -10.297 -6.829 1.00 50.90 S \nATOM 8 CE MET A 1 4.342 -11.767 -7.798 1.00 45.05 C \nATOM 9 N SER A 2 3.944 -6.105 -10.433 1.00 64.45 N \n

In [ ]:
df

,id,nmsa,pLDDT_AlphaFold,version,sequence,length,pLDDT_before,pLDDT_after,tm_score_before,lddt_before,tm_score_after,lddt_after,time,difference_plddt
0,A0A0H4SQ70,10124,91.406506,BASE,MQSIDFGRALDFTLRWEGGYVNHPSDPGGATNRGITQVTYNQWRTQKGLPTREVRLIEEDEVRSIYWQFYWAPVEGRTAPSWVQFRVCLFDTFVQFGVFGGTFLWQKVCGVPADGQWGPVTSRATENLVSTKGPLWSGMALVGERVRYRAQRVSQNRSQLAFLQGWLNRDSDLLLYLLNLR,181,85.136031,87.117266,0.98978,0.990052,0.98613,0.981236,67.120121,1.981235
1,Q80B01,5665,91.960021,BASE,ENRAFDENVTERVMHVLVAAGADVNAASVVDRTPLHVCLTGMSTHPGTIAALLRFGADVNAVDLCGMSPLAVLVRSRAATAELVRMLLDAGADAHAVDSRLDSLLHQHFQSARPRPEVVRELIRHGCSPRARNRIGNTPLHEAAKHSSCKHSLVGPLLAAGASVDARNNTGRTPLHLAAASNPRACRRLIALGADVVARSYAGVTPLAQLIADNNSALVTAALDTQPEPRAVAESLRATTPVGETACSRLCVAYVVARAPSEVLGEPERALHAAFVAECLAEVAAIHAVRCGTPPVSLLEILVSARTPRSLLSRRARRLAESRTTVYRAPLRARIAAMRHRSRLVERALRTLRGCVLPREVLERVLRCLSTQDLRASGLAE,381,90.829035,90.912645,0.88055,0.896696,0.88207,0.897574,0.639445,0.083609
2,A0A1S5R1Q9,57,94.034389,BASE+LOGAN,MSLDIPKGMNIHSAATYARDYAAKQRANYVTIVFNDINLTVYATSNPDDIALIYYLESEVRRLKR,65,58.079447,83.771889,0.50373,0.623064,0.67245,0.756843,34.663036,25.692443
3,A0A2I4Q1U7,17,91.027945,BASE+LOGAN,MNYCDIAHELRMEREKQEKRIIKKMAVLLAHYKADKQPTHDEFVDFCNMYLNVSKATGYRWLKALNDGEL,70,78.155634,87.247534,0.60368,0.743309,0.94927,0.969441,35.922151,9.091901
4,A0A218MN90,538,90.865686,BASE,METFVGFVLYLYTSAGTLLEFTPKDSLSDCLRIKRTIERTDPPQRGKERWVCKQGKLKLKVIDGKKYPVEVLDY,74,81.071421,81.071421,0.95585,0.950043,0.95585,0.950043,34.210736,0.000000
5,A0A3S7URH3,1016,90.055809,BASE,MPHEIWGQRFVSTREPAWHRIGFHFAEPILPSEAVAIAGFEEPELAPLFLEDGTPVDYYAVVMGDEVFGVVSRYWRLFRLEEIVPTLDELSRTFPLSAAGQLKKGRIVFFAFEQRTEILGEDYIRYLVVLHSYEPGRSWKVLYTPTRVVCMNTLVASEKDREWEYRVYHNAPRGILEAQIVMAQYRVLQETVDKKLEAFARIGDFDAQLDVLFEHVYPYPEPPSELDKQRYGKEVEERYERAMERVKQVRNLALQSYQRFNDEFPKFGNTAYAALQAVTEVADWRGNLKSKESPLVGPRAKEKQAAWQFLSTLI,314,77.239656,77.239656,0.48237,0.768121,0.48237,0.768121,160.641790,0.000000
6,A0A5B9NC75,582,91.237163,BASE,MVSSDFFEADDLFIMDSYAFGDRGILMSQDKDSWLSPMARFDIPTGTVWPALDNPFGWIKWDDTQAMPVRAHGLKFFWWQMLAGDDADNVKGITLLDGKPCGKRTAFDAIYPITSEQDAAEFVVAAYARNNQDVLAEAECLFLRRSQSDSAYQYLMSLLTTPSLRDWVQSLHEYHKQHIQWIQEHPDNGENDS,193,87.290006,90.582240,0.97195,0.940903,0.98274,0.969921,69.603494,3.292234
7,Q5GQY6,718,90.340547,BASE+LOGAN+12CY,MKINFKSHQITDREVDFTQDDQVRLFEIMKQEFINHITYSRFDSIIHLTKFEQQLYNFCKDYNVDIEYSKDRVAFFTALIKEMKYQ,86,57.228919,60.270521,0.39317,0.649580,0.39527,0.648882,35.569681,3.041602
8,K8DUZ9,123,90.277649,BASE,MNAFEKRAQLKDIKPGAILYEVFSINGVKAEMGPKKIITGLPFQHLSIGLFVDAITVYDDWEGRQHMSLMDHNVIGRNNYNFHALFLSKKDAQEYVDQINNDQLPPELRETSRKMHREWIVRRAEDALYDM,131,67.014851,73.684647,0.78825,0.733380,0.88960,0.899537,44.113523,6.669796
9,A0A0A0RPE0,30,90.647829,BASE+LOGAN+12CY,MLILIMMVVAVVGPLVALIVTEVCDAKWDRKYNEVMEQRYAERRVRKWEQRYNAHSDTLAS,61,74.841414,74.841414,0.87703,1.000000,0.87703,1.000000,34.537524,0.000000


In [11]:
import ast

# First, load the log file, as it is defined in the cell below
log = pd.read_csv(base_path / 'logs_df' / 'A0A1S5R1Q9_log.tsv', sep="\t")
# The 'pdb' column is a string representation of a list.


Path('./t_df.pdb').write_text(log.iloc[0].pdb)

42486

In [16]:
from proteinttt.utils.structure import calculate_tm_score, lddt_score

lddt_score(Path('/scratch/project/open-35-8/pimenol1/ProteinTTT/ProteinTTT/notebooks/t_df.pdb'), Path('/scratch/project/open-35-8/pimenol1/ProteinTTT/ProteinTTT/data/bfvd/predicted_structures/A0A1S5R1Q9.pdb'))

0.9319476176082502

In [17]:
log = pd.read_csv(base_path / 'logs_msa' / 'A0A0H4SQ70_log.tsv', sep="\t")
log.head(10)

,step,accumulated_step,loss,eval_step_time,plddt,pdb
0,0,0,NaN,66.267741,49.100296,['PARENT N/A\nATOM 1 N TYR A 521 -12.885 -18.125 12.595 1.00 54.20 N \nATOM 2 CA TYR A 521 -11.848 -17.834 11.612 1.00 48.39 C \nATOM 3 C TYR A 521 -10.808 -16.877 12.183 1.00 52.31 C \nATOM 4 CB TYR A 521 -12.463 -17.238 10.342 1.00 46.73 C \nATOM 5 O TYR A 521 -9.741 -16.688 11.595 1.00 50.11 O \nATOM 6 CG TYR A 521 -11.487 -17.109 9.198 1.00 43.25 C \nATOM 7 CD1 TYR A 521 -10.830 -15.905 8.949 1.00 38.72 C \nATOM 8 CD2 TYR A 521 -11.219 -18.188 8.363 1.00 45.23 C \nATOM 9 CE1 TYR A 521 -9.931 -15.780 7.895 1.00 51.69 C \nATOM 10 CE2 TYR A 521 -10.322 -18.075 7.306 1.00 53.00 C \nATOM 11 OH TYR A 521 -8.794 -16.751 6.036 1.00 63.32 O \nATOM 12 CZ TYR A 521 -9.683 -16.869 7.081 1.00 35.15 C ...
1,1,1,1.710938,64.769179,43.330227,['PARENT N/A\nATOM 1 N TYR A 521 8.269 -8.280 4.499 1.00 49.75 N \nATOM 2 CA TYR A 521 7.403 -7.114 4.635 1.00 49.37 C \nATOM 3 C TYR A 521 6.092 -7.486 5.317 1.00 49.05 C \nATOM 4 CB TYR A 521 7.119 -6.492 3.264 1.00 44.05 C \nATOM 5 O TYR A 521 5.208 -8.081 4.696 1.00 46.02 O \nATOM 6 CG TYR A 521 6.592 -5.080 3.336 1.00 41.38 C \nATOM 7 CD1 TYR A 521 5.222 -4.826 3.339 1.00 40.29 C \nATOM 8 CD2 TYR A 521 7.462 -3.997 3.399 1.00 44.29 C \nATOM 9 CE1 TYR A 521 4.732 -3.526 3.403 1.00 54.64 C \nATOM 10 CE2 TYR A 521 6.983 -2.692 3.464 1.00 50.04 C \nATOM 11 OH TYR A 521 5.138 -1.179 3.528 1.00 58.87 O \nATOM 12 CZ TYR A 521 5.618 -2.468 3.465 1.00 55.88 C ...
2,2,2,1.480469,64.768631,57.404964,['PARENT N/A\nATOM 1 N TYR A 521 9.223 7.274 -18.997 1.00 63.40 N \nATOM 2 CA TYR A 521 8.070 6.840 -18.216 1.00 62.27 C \nATOM 3 C TYR A 521 8.237 5.397 -17.753 1.00 61.62 C \nATOM 4 CB TYR A 521 6.782 6.979 -19.033 1.00 56.85 C \nATOM 5 O TYR A 521 8.093 4.464 -18.546 1.00 57.25 O \nATOM 6 CG TYR A 521 5.524 6.810 -18.217 1.00 53.65 C \nATOM 7 CD1 TYR A 521 4.880 5.576 -18.142 1.00 50.67 C \nATOM 8 CD2 TYR A 521 4.975 7.881 -17.521 1.00 52.27 C \nATOM 9 CE1 TYR A 521 3.719 5.416 -17.393 1.00 58.90 C \nATOM 10 CE2 TYR A 521 3.815 7.732 -16.769 1.00 55.21 C \nATOM 11 OH TYR A 521 2.046 6.344 -15.968 1.00 65.76 O \nATOM 12 CZ TYR A 521 3.195 6.497 -16.711 1.00 53.57 C ...
3,3,3,1.748047,64.912733,47.754387,['PARENT N/A\nATOM 1 N TYR A 521 -0.202 -13.613 1.488 1.00 62.93 N \nATOM 2 CA TYR A 521 -0.898 -12.397 1.894 1.00 62.07 C \nATOM 3 C TYR A 521 -0.147 -11.156 1.425 1.00 63.77 C \nATOM 4 CB TYR A 521 -1.071 -12.359 3.415 1.00 54.88 C \nATOM 5 O TYR A 521 0.887 -10.798 1.993 1.00 57.21 O \nATOM 6 CG TYR A 521 -2.081 -11.341 3.886 1.00 52.41 C \nATOM 7 CD1 TYR A 521 -3.443 -11.633 3.894 1.00 54.49 C \nATOM 8 CD2 TYR A 521 -1.676 -10.084 4.323 1.00 58.96 C \nATOM 9 CE1 TYR A 521 -4.377 -10.697 4.325 1.00 66.40 C \nATOM 10 CE2 TYR A 521 -2.601 -9.140 4.756 1.00 63.30 C \nATOM 11 OH TYR A 521 -4.868 -8.525 5.182 1.00 67.78 O \nATOM 12 CZ TYR A 521 -3.947 -9.456 4.754 1.00 69.11 C ...
4,4,4,1.652344,64.854294,52.955860,['PARENT N/A\nATOM 1 N TYR A 521 1.727 -17.393 23.501 1.00 47.78 N \nATOM 2 CA TYR A 521 2.962 -16.982 22.842 1.00 46.77 C \nATOM 3 C TYR A 521 2.720 -16.684 21.367 1.00 46.27 C \nATOM 4 CB TYR A 521 4.034 -18.066 22.986 1.00 44.43 C \nATOM 5 O TYR A 521 2.548 -17.602 20.562 1.00 44.73 O \nATOM 6 CG TYR A 521 5.432 -17.580 22.691 1.00 41.91 C \nATOM 7 CD1 TYR A 521 5.975 -17.693 21.413 1.00 38.82 C \nATOM 8 CD2 TYR A 521 6.214 -17.008 23.689 1.00 40.24 C \nATOM 9 CE1 TYR A 521 7.263 -17.248 21.136 1.00 42.26 C \nATOM 10 CE2 TYR A 521 7.504 -16.559 23.424 1.00 44.53 C \nATOM 11 OH TYR A 521 9.295 -16.241 21.879 1.00 43.16 O \nATOM 12 CZ TYR A 521 8.019 -16.683 22.147 1.00 39.58 C ...
5,5,5,1.800781,64.848545,64.168854,['PARENT N/A\nATOM 1 N TYR A 521 -16.536 0.929 13.565 1.00 66.82 N \nATOM 2 CA TYR A 521 -17.047 0.318 14.787 1.00 65.09 C \nATOM 3 C TYR A 521 -17.727 -1.013 14.489 1.00 65.53 C \nATOM 4 CB TYR A 521 -18.030 1.261 15.488 1.00 59.19 C \nATOM 5 O TYR A 521 -18.921 -1.053 14.185 1.00 60.93 O \nATOM 6 CG TYR A 

In [40]:
log = pd.read_csv(base_path / 'logs_df' / 'A0A0H4SQ70_log.tsv', sep="\t")
log

,step,accumulated_step,loss,score_seq_time,eval_step_time,plddt,pdb
0,0,0,NaN,0.0,1.979763,85.136031,PARENT N/A\nATOM 1 N MET A 1 -6.334 -13.737 14.853 1.00 30.43 N \nATOM 2 CA MET A 1 -5.347 -14.366 13.981 1.00 40.25 C \nATOM 3 C MET A 1 -5.152 -13.556 12.704 1.00 31.69 C \nATOM 4 CB MET A 1 -5.768 -15.795 13.634 1.00 30.67 C \nATOM 5 O MET A 1 -6.104 -13.335 11.953 1.00 31.98 O \nATOM 6 CG MET A 1 -4.754 -16.851 14.044 1.00 30.73 C \nATOM 7 SD MET A 1 -5.402 -18.557 13.861 1.00 18.27 S \nATOM 8 CE MET A 1 -4.304 -19.179 12.557 1.00 18.19 C \nATOM 9 N GLN A 2 -4.185 -12.569 12.812 1.00 53.99 N \nATOM 10 CA GLN A 2 -3.840 -11.817 11.610 1.00 55.16 C \nATOM 11 C GLN A 2 -3.514 -12.753 10.450 1.00 55.60 C \nATOM 12 CB GLN A 2 -2.659 -10.884 11.881 1.00 46.79 C \n...
1,1,4,1.396484,0.0,1.963782,84.821798,PARENT N/A\nATOM 1 N MET A 1 -6.324 -14.365 14.345 1.00 31.27 N \nATOM 2 CA MET A 1 -5.386 -14.990 13.416 1.00 40.29 C \nATOM 3 C MET A 1 -5.192 -14.126 12.175 1.00 32.28 C \nATOM 4 CB MET A 1 -5.875 -16.382 13.013 1.00 30.81 C \nATOM 5 O MET A 1 -6.154 -13.830 11.463 1.00 32.54 O \nATOM 6 CG MET A 1 -4.909 -17.499 13.374 1.00 30.87 C \nATOM 7 SD MET A 1 -5.617 -19.165 13.075 1.00 18.64 S \nATOM 8 CE MET A 1 -4.472 -19.777 11.807 1.00 20.03 C \nATOM 9 N GLN A 2 -4.170 -13.184 12.296 1.00 54.12 N \nATOM 10 CA GLN A 2 -3.833 -12.368 11.134 1.00 55.47 C \nATOM 11 C GLN A 2 -3.538 -13.240 9.917 1.00 55.73 C \nATOM 12 CB GLN A 2 -2.634 -11.468 11.439 1.00 46.81 C \n...
2,2,8,1.220703,0.0,1.963898,85.219783,PARENT N/A\nATOM 1 N MET A 1 -7.434 -14.632 13.660 1.00 31.07 N \nATOM 2 CA MET A 1 -6.526 -15.275 12.714 1.00 40.43 C \nATOM 3 C MET A 1 -6.271 -14.377 11.508 1.00 32.44 C \nATOM 4 CB MET A 1 -7.091 -16.620 12.255 1.00 31.00 C \nATOM 5 O MET A 1 -7.206 -14.005 10.797 1.00 32.61 O \nATOM 6 CG MET A 1 -6.252 -17.815 12.679 1.00 31.03 C \nATOM 7 SD MET A 1 -7.061 -19.415 12.291 1.00 18.34 S \nATOM 8 CE MET A 1 -5.883 -20.086 11.085 1.00 18.67 C \nATOM 9 N GLN A 2 -5.231 -13.457 11.706 1.00 54.60 N \nATOM 10 CA GLN A 2 -4.829 -12.606 10.591 1.00 55.89 C \nATOM 11 C GLN A 2 -4.523 -13.438 9.348 1.00 56.10 C \nATOM 12 CB GLN A 2 -3.610 -11.763 10.970 1.00 47.16 C \n...
3,3,12,1.214844,0.0,1.965569,85.962069,PARENT N/A\nATOM 1 N MET A 1 -9.782 -11.386 10.232 1.00 51.29 N \nATOM 2 CA MET A 1 -8.379 -11.026 10.046 1.00 51.52 C \nATOM 3 C MET A 1 -7.554 -12.243 9.642 1.00 53.52 C \nATOM 4 CB MET A 1 -7.808 -10.407 11.323 1.00 44.64 C \nATOM 5 O MET A 1 -6.790 -12.776 10.449 1.00 50.50 O \nATOM 6 CG MET A 1 -8.360 -9.025 11.634 1.00 41.71 C \nATOM 7 SD MET A 1 -7.713 -8.348 13.212 1.00 46.27 S \nATOM 8 CE MET A 1 -9.213 -7.560 13.861 1.00 36.66 C \nATOM 9 N GLN A 2 -8.061 -13.139 8.778 1.00 62.85 N \nATOM 10 CA GLN A 2 -7.582 -14.484 8.478 1.00 65.31 C \nATOM 11 C GLN A 2 -7.247 -14.631 6.997 1.00 64.09 C \nATOM 12 CB GLN A 2 -8.621 -15.528 8.888 1.00 51.83 C \n...
4,4,16,0.666016,0.0,1.964066,86.218772,PARENT N/A\nATOM 1 N MET A 1 -10.276 -12.599 9.095 1.00 50.67 N \nATOM 2 CA MET A 1 -8.922 -12.095 8.887 1.00 51.52 C \nATOM 3 C MET A 1 -7.978 -13.221 8.480 1.00 52.82 C \nATOM 4 CB MET A 1 -8.403 -11.409 10.153 1.00 44.66 C \nATOM 5 O MET A 1 -7.457 -13.940 9.335 1.00 50.02 O \nATOM 6 CG MET A 1 -9.052 -10.063 10.431 1.00 42.26 C \nATOM 7 SD MET A 1 -8.471 -9.314 12.002 1.00 45.40 S \nATOM 8 CE MET A 1 -9.946 -8.366 12.469 1.00 37.49 C \nATOM 9 N GLN A 2 -8.307 -14.075 7.503 1.00 62.04 N \nATOM 10 CA GLN A 2 -7.733 -15.371 7.158 1.00 64.13 C \nATOM 11 C GLN A 2 -7.245 -15.390 5.712 1.00 63.07 C \nATOM 12 CB GLN A 2 -8.755 -16.487 7.381 1.00 51.24 C \n...
5,5,20,0.564453,0.0,1.964790,86.328141,PARENT N/A\nATOM 1 N MET A 1 -10.722 -11.004 10.500 1.00 52.56 N \nATOM 2 CA MET A 1 -9.354 -10.683 10.102 1.00 53.59 C \nATOM 3 C MET A 1 -8.597 -11.942 9.694 1.00 54.92 C \nATOM 4 CB MET A 1 -8.616 -9.974 11.238 1.00 46.18 C \nATOM 5 O MET A 1 -8.024 -12.630 10.541 1.00 51.37 O \nATOM 6 CG MET A 1 -9.124 -8.569 11.517 1.00 43.48 C \nATOM 7 SD MET A 1 

In [3]:
from Bio import SeqIO


def check_a3m_lengths(filepath):
    lengths = {}
    try:
        for record in SeqIO.parse(filepath, format="fasta"):
            # DELETE all lowercase letters from the sequence
            for char in record.seq:
                if char.islower():
                    record.seq = record.seq.replace(char, "")
            lengths[record.id] = len(record.seq)

        first_record_id = list(lengths.keys())[0]
        reference_length = lengths[first_record_id]

        print(f"Reference sequence: '{first_record_id}' (Length: {reference_length})")

        is_consistent = True
        for seq_id, length in lengths.items():
            if length != reference_length:
                print(f"MISMATCH: '{seq_id}' has length {length}")
                is_consistent = False

        if is_consistent:
            print(f"All {len(lengths)} sequences have the same length: {reference_length}")

    except Exception as e:
        print(f"An error occurred: {e}")

file_to_check = "/scratch/project/open-35-8/antonb/bfvd/bfvd_msa/A0A1M7XUY2.a3m"
check_a3m_lengths(file_to_check)

Reference sequence: 'UniRef100_A0A1M7XUY2' (Length: 437)
All 2527 sequences have the same length: 437
